<a href="https://colab.research.google.com/github/IntelAntique/BLEsensor/blob/main/Health.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split, Dataset
import numpy as np
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
df = pd.read_csv('Diabetes-Data.csv', index_col = False)
df = df.drop(columns="Id")
n = len(df.columns)
dataset = df.values

train, test = torch.utils.data.random_split(dataset, [0.8, 0.2])

In [3]:
training_data = torch.tensor(train)
test_data = torch.tensor(test)

# print(training_data[:, n - 1: n])
# print(training_data[:, 0: n - 1])

<ipython-input-3-baf1f0835146>:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  training_data = torch.tensor(train)


In [4]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for data in test_dataloader:
    X, y = data[:, : n - 1], data[:, n - 1]
    print(f"Shape of X [N, C]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C]: torch.Size([64, 8])
Shape of y: torch.Size([64]) torch.float64


In [5]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [6]:
input = 8
output = 2

class BinaryClassification(nn.Module):
    def __init__(self, input, output):
        super().__init__()
        self.sequential = nn.Sequential(
            nn.Linear(input, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, output)
        )

    def forward(self, x):
        return self.sequential(x)

model = BinaryClassification(input, output).to(device)

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, data in enumerate(dataloader):
        X, y = data[:, : n - 1].type(torch.float).to(device), data[:, n - 1].type(torch.long).to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 10 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for data in dataloader:
            X, y = data[:, : n - 1].type(torch.float).to(device), data[:, n - 1].type(torch.long).to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [10]:
epochs = 1000
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Streaming output truncated to the last 5000 lines.
loss: 0.029123  [ 1984/ 2215]
Test Error: 
 Accuracy: 98.2%, Avg loss: 0.105736 

Epoch 446
-------------------------------
loss: 0.008518  [   64/ 2215]
loss: 0.038466  [  704/ 2215]
loss: 0.036370  [ 1344/ 2215]
loss: 0.025118  [ 1984/ 2215]
Test Error: 
 Accuracy: 98.2%, Avg loss: 0.108397 

Epoch 447
-------------------------------
loss: 0.008190  [   64/ 2215]
loss: 0.037874  [  704/ 2215]
loss: 0.034950  [ 1344/ 2215]
loss: 0.025915  [ 1984/ 2215]
Test Error: 
 Accuracy: 98.2%, Avg loss: 0.106597 

Epoch 448
-------------------------------
loss: 0.007152  [   64/ 2215]
loss: 0.038139  [  704/ 2215]
loss: 0.035835  [ 1344/ 2215]
loss: 0.025077  [ 1984/ 2215]
Test Error: 
 Accuracy: 98.2%, Avg loss: 0.105890 

Epoch 449
-------------------------------
loss: 0.006843  [   64/ 2215]
loss: 0.036105  [  704/ 2215]
loss: 0.032357  [ 1344/ 2215]
loss: 0.024502  [ 1984/ 2215]
Test Error: 
 Accuracy: 98.2%, Avg loss: 0.104662 

Epoch 450
-